# Retail Performance Analysis (BA/DA Report)
This notebook consolidates key business analytics insights from the Alberta retail data mart.

## 0. Setup

In [1]:
import pandas as pd
import plotly.express as px

from src.data_pipeline import (
    list_available_months,
    load_and_prepare_data,
    load_reference_tables,
    build_promotion_summary,
    build_profitability_by_store,
)
from src.report_utils import (
    monthly_kpi_summary,
    category_revenue_share,
    promotion_activity_timeline,
    store_opening_trend,
    web_account_share,
    prepare_transaction_level,
    run_promo_regression,
    ab_test_promo,
)


In [2]:
import importlib
import src.report_utils

importlib.reload(src.report_utils)


<module 'src.report_utils' from '/Users/yuanhaowen/Desktop/Project/SaleAnalysisApp/src/report_utils.py'>

## 1. Load curated datasets

In [3]:
MONTHS = list_available_months()  # load all available months by default
MONTHS = MONTHS[:12]
MONTHS


('202301',
 '202302',
 '202303',
 '202304',
 '202305',
 '202306',
 '202307',
 '202308',
 '202309',
 '202310',
 '202311',
 '202312')

In [4]:
sales_df = load_and_prepare_data(months=MONTHS)
reference_tables = load_reference_tables()
contact_df = reference_tables['contact']
promotion_df = reference_tables['promotion']
store_df = reference_tables['store']
sales_df.shape


(12000, 57)

## 2. Sales activity overview

In [5]:
monthly_summary = monthly_kpi_summary(sales_df)
monthly_summary.head()

,Month,Revenue,Orders,Units,Unique_Customers,AvgOrderValue
0,2023-01-01,83749.15,992,873,164,84.424546
1,2023-02-01,88319.91,994,886,171,88.853028
2,2023-03-01,88236.11,996,916,172,88.590472
3,2023-04-01,89881.06,993,947,206,90.514663
4,2023-05-01,89879.95,991,949,181,90.696216


In [6]:
fig = px.line(monthly_summary, x='Month', y='Revenue', markers=True, title='Monthly Revenue Trend')
fig.update_layout(xaxis_title='Month', yaxis_title='Revenue')
fig


/Users/yuanhaowen/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [7]:
fig_orders = px.bar(monthly_summary, x='Month', y='Orders', title='Order Volume by Month')
fig_orders.update_layout(xaxis_title='Month', yaxis_title='Orders')
fig_orders


/Users/yuanhaowen/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



**Observation:** Revenue peaks in spring/summer months while order volumes remain comparatively stable, signalling higher average baskets during promotional events.

## 3. Merchandise mix

In [8]:
category_share = category_revenue_share(sales_df, top_n=10)
category_share


,Catégorie,Revenue,Share
0,Catégorie_006,228227.22,0.274855
1,Catégorie_007,213887.15,0.257585
2,Catégorie_002,168392.20,0.202795
3,Catégorie_003,90678.09,0.109204
4,Catégorie_004,69075.14,0.083187
5,Catégorie_001,36295.55,0.043711
6,Catégorie_005,22874.53,0.027548
7,Catégorie_008,925.10,0.001114


In [9]:
fig_cat = px.bar(category_share, x='Catégorie', y='Revenue', title='Top Categories by Revenue')
fig_cat.update_layout(xaxis_title='Category', yaxis_title='Revenue', xaxis_tickangle=45)
fig_cat


Seasonal collections dominate revenue share, with Spring assortments consistently outperforming Fall launches.

### Category performance (L1) – Pareto view


In [10]:
from IPython.display import display

category_perf = (
    sales_df.assign(Category=sales_df['Catégorie'].fillna('Unknown'))
    .groupby('Category', as_index=False)
    .agg(
        GMV=('Transaction_value', 'sum'),
        Units=('Quantity_item', 'sum'),
        Orders=('Transaction_ID', 'nunique'),
    )
)

category_perf['ASP'] = category_perf['GMV'] / category_perf['Units'].replace({0: pd.NA})
total_gmv = category_perf['GMV'].sum()
category_perf['GMV_share_pct'] = (category_perf['GMV'] / total_gmv) * 100
category_perf = category_perf.sort_values('GMV', ascending=False).reset_index(drop=True)
category_perf['GMV_cum_share_pct'] = category_perf['GMV_share_pct'].cumsum()
category_perf['Pareto_80'] = category_perf['GMV_cum_share_pct'] <= 80
category_columns = ['Category', 'GMV', 'Units', 'Orders', 'ASP', 'GMV_share_pct', 'GMV_cum_share_pct', 'Pareto_80']
display(category_perf.loc[:, category_columns].head(15))
display(category_perf.loc[category_perf['Pareto_80'], category_columns])


,Category,GMV,Units,Orders,ASP,GMV_share_pct,GMV_cum_share_pct,Pareto_80
0,Unknown,229604.91,2536,2526,90.538214,21.661660,21.661660,True
1,Catégorie_006,228227.22,3024,2989,75.471964,21.531685,43.193345,True
2,Catégorie_007,213887.15,2518,2776,84.943268,20.178797,63.372141,True
3,Catégorie_002,168392.20,1240,1474,135.800161,15.886658,79.258799,True
4,Catégorie_003,90678.09,777,880,116.702819,8.554861,87.813660,False
5,Catégorie_004,69075.14,632,748,109.296108,6.516769,94.330429,False
6,Catégorie_001,36295.55,282,331,128.707624,3.424238,97.754667,False
7,Catégorie_005,22874.53,182,221,125.684231,2.158056,99.912723,False
8,Catégorie_008,925.10,9,9,102.788889,0.087277,100.000000,False


,Category,GMV,Units,Orders,ASP,GMV_share_pct,GMV_cum_share_pct,Pareto_80
0,Unknown,229604.91,2536,2526,90.538214,21.661660,21.661660,True
1,Catégorie_006,228227.22,3024,2989,75.471964,21.531685,43.193345,True
2,Catégorie_007,213887.15,2518,2776,84.943268,20.178797,63.372141,True
3,Catégorie_002,168392.20,1240,1474,135.800161,15.886658,79.258799,True


### Top 50 SKUs – GMV concentration


In [11]:
sku_perf = (
    sales_df.assign(
        SKU_Label=sales_df['Style - Description FR']
        .fillna(sales_df['SKU'])
        .fillna(sales_df['Item_ID'])
    )
    .groupby(['SKU', 'SKU_Label'], as_index=False)
    .agg(
        GMV=('Transaction_value', 'sum'),
        Units=('Quantity_item', 'sum'),
        Orders=('Transaction_ID', 'nunique'),
    )
)

sku_perf['ASP'] = sku_perf['GMV'] / sku_perf['Units'].replace({0: pd.NA})
sku_total_gmv = sku_perf['GMV'].sum()
sku_perf['GMV_share_pct'] = (sku_perf['GMV'] / sku_total_gmv) * 100
sku_perf = sku_perf.sort_values('GMV', ascending=False).reset_index(drop=True)
sku_perf['GMV_cum_share_pct'] = sku_perf['GMV_share_pct'].cumsum()
sku_perf['Pareto_80'] = sku_perf['GMV_cum_share_pct'] <= 80
sku_columns = ['SKU', 'SKU_Label', 'GMV', 'Units', 'Orders', 'ASP', 'GMV_share_pct', 'GMV_cum_share_pct', 'Pareto_80']
top_skus = sku_perf.loc[:, sku_columns].head(50)
display(top_skus)
display(top_skus[top_skus['Pareto_80']])


,SKU,SKU_Label,GMV,Units,Orders,ASP,GMV_share_pct,GMV_cum_share_pct,Pareto_80
0,SKUBD9E38FA,SKUBD9E38FA,229604.91,2536,2526,90.538214,21.661660,21.661660,True
1,SKU0D7415C7,Style_6380,2361.89,30,30,78.729667,0.222828,21.884488,True
2,SKUBD1DDE57,Style_6379,1815.30,25,24,72.612,0.171261,22.055750,True
3,SKU5B935F42,Style_3356,1737.95,2,2,868.975,0.163964,22.219713,True
4,SKU3EF8334F,Style_7250,1536.40,10,10,153.64,0.144949,22.364662,True
5,SKU9F5059EB,Style_6642,956.49,1,1,956.49,0.090238,22.454901,True
6,SKUEABB9A7C,Style_6381,898.24,12,12,74.853333,0.084743,22.539643,True
7,SKU64FE8662,Style_5472,880.96,4,4,220.24,0.083113,22.622756,True
8,SKUD540B6DD,Style_2505,878.00,4,4,219.5,0.082833,22.705589,True
9,SKU284580DB,Style_3576,854.07,2,2,427.035,0.080576,22.786165,True


,SKU,SKU_Label,GMV,Units,Orders,ASP,GMV_share_pct,GMV_cum_share_pct,Pareto_80
0,SKUBD9E38FA,SKUBD9E38FA,229604.91,2536,2526,90.538214,21.661660,21.661660,True
1,SKU0D7415C7,Style_6380,2361.89,30,30,78.729667,0.222828,21.884488,True
2,SKUBD1DDE57,Style_6379,1815.30,25,24,72.612,0.171261,22.055750,True
3,SKU5B935F42,Style_3356,1737.95,2,2,868.975,0.163964,22.219713,True
4,SKU3EF8334F,Style_7250,1536.40,10,10,153.64,0.144949,22.364662,True
5,SKU9F5059EB,Style_6642,956.49,1,1,956.49,0.090238,22.454901,True
6,SKUEABB9A7C,Style_6381,898.24,12,12,74.853333,0.084743,22.539643,True
7,SKU64FE8662,Style_5472,880.96,4,4,220.24,0.083113,22.622756,True
8,SKUD540B6DD,Style_2505,878.00,4,4,219.5,0.082833,22.705589,True
9,SKU284580DB,Style_3576,854.07,2,2,427.035,0.080576,22.786165,True


## 4. Promotion lifecycle and impact

In [12]:
promotion_summary = build_promotion_summary(sales_df)
promotion_summary

,orders,revenue,avg_order_value,total_discount,avg_discount_per_order,units
Promotion,,,,,,
No Promotion,7617,710557.84,93.285787,2686.04,0.352638,6826
With Promotion,4321,349402.05,80.861386,38405.84,8.888183,4374


In [13]:
promotion_timeline = promotion_activity_timeline(promotion_df)
promotion_timeline.head()

/Users/yuanhaowen/Desktop/Project/SaleAnalysisApp/src/report_utils.py:47: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,Month,ActivePromotions
0,2021-04-01,1
1,2021-05-01,1
2,2021-06-01,1
3,2021-07-01,1
4,2021-08-01,1


In [14]:
fig_promo = px.line(promotion_timeline, x='Month', y='ActivePromotions', title='Active Promotions Over Time')
fig_promo.update_layout(xaxis_title='Month', yaxis_title='Active Promotions')
fig_promo.update_xaxes(range=[promotion_timeline['Month'].min(), pd.Timestamp.today() + pd.DateOffset(years=1)])
fig_promo


/Users/yuanhaowen/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### Promotion effectiveness – promo vs non-promo


In [15]:
from IPython.display import display

transaction_level = (
    sales_df.groupby('Transaction_ID', as_index=False)
    .agg(
        GMV=('Line_Sales_Value', 'sum'),
        Units=('Quantity_item', 'sum'),
        Discount_value=('Line_Discount', 'sum'),
        Original_value=('Line_Original_Value', 'sum'),
        Promo_Flag=('Promo_Flag', 'max'),
    )
)

transaction_level['Orders'] = 1

promo_overall = (
    transaction_level
    .groupby('Promo_Flag', as_index=False)
    .agg(
        GMV=('GMV', 'sum'),
        Units=('Units', 'sum'),
        Orders=('Orders', 'sum'),
        Discount_value=('Discount_value', 'sum'),
        Original_value=('Original_value', 'sum'),
    )
)

promo_overall['ASP'] = promo_overall['GMV'] / promo_overall['Units'].replace({0: pd.NA})
promo_overall['AOV'] = promo_overall['GMV'] / promo_overall['Orders'].replace({0: pd.NA})
promo_overall['Avg_Discount_rate'] = promo_overall['Discount_value'] / promo_overall['Original_value'].replace({0: pd.NA})
promo_overall['GMV_share_pct'] = (promo_overall['GMV'] / promo_overall['GMV'].sum()) * 100
promo_overall = promo_overall[
    ['Promo_Flag', 'GMV', 'Units', 'Orders', 'ASP', 'AOV', 'Discount_value', 'Avg_Discount_rate', 'GMV_share_pct']
]

display(promo_overall)

analysis_base = sales_df.assign(
    Category=sales_df['Catégorie'].fillna('Unknown'),
    Store_Name=sales_df['LIBELLE_y'].fillna(sales_df['Store_transaction_ID']),
    Store_City=sales_df['CITY'].fillna('Unknown'),
    Store_Province=sales_df['PROVINCE'].fillna('Unknown'),
    Promo_Label=sales_df['LIBELLE_x'].fillna('Unspecified promo'),
)
analysis_base['Original_value'] = analysis_base['Line_Original_Value']
analysis_base['Discount_value'] = analysis_base['Line_Discount'].clip(lower=0)
analysis_base['GMV'] = analysis_base['Line_Sales_Value']


,Promo_Flag,GMV,Units,Orders,ASP,AOV,Discount_value,Avg_Discount_rate,GMV_share_pct
0,False,200674.01,6816,7607,29.441609,26.380177,2678.57,0.013172,84.013452
1,True,38185.37,4384,4321,8.710167,8.837160,38413.31,0.501488,15.986548


#### Category lens


In [16]:
from IPython.display import display

def summarize_promo_effect(df, dimensions):
    grouped = (
        df.groupby(dimensions + ['Promo_Flag'], as_index=False)
        .agg(
            GMV=('GMV', 'sum'),
            Units=('Quantity_item', 'sum'),
            Orders=('Transaction_ID', 'nunique'),
            Discount_value=('Discount_value', 'sum'),
            Original_value=('Original_value', 'sum'),
        )
    )
    grouped['ASP'] = grouped['GMV'] / grouped['Units'].replace({0: pd.NA})
    grouped['AOV'] = grouped['GMV'] / grouped['Orders'].replace({0: pd.NA})
    grouped['Avg_Discount_rate'] = grouped['Discount_value'] / grouped['Original_value'].replace({0: pd.NA})

    def share(values):
        total = values.sum()
        if total == 0:
            return pd.Series([0.0] * len(values), index=values.index)
        return (values / total) * 100

    grouped['GMV_share_pct'] = grouped.groupby(dimensions)['GMV'].transform(share)
    return grouped.sort_values('GMV', ascending=False)

category_promo = summarize_promo_effect(analysis_base, ['Category'])
display(category_promo.head(20))


,Category,Promo_Flag,GMV,Units,Orders,Discount_value,Original_value,ASP,AOV,Avg_Discount_rate,GMV_share_pct
16,Unknown,False,50249.66,1654,1647,1580.81,51830.47,30.380689,30.509812,0.030500,87.476379
2,Catégorie_002,False,44214.69,1043,1278,0.00,44214.69,42.391841,34.596784,0.000000,94.344839
12,Catégorie_007,False,42660.54,1791,2059,0.00,42660.54,23.819397,20.719058,0.000000,81.815640
4,Catégorie_003,False,22206.40,559,662,0.00,22206.40,39.725224,33.544411,0.000000,84.434307
6,Catégorie_004,False,16630.54,487,604,0.00,16630.54,34.148953,27.534007,0.000000,88.035221
10,Catégorie_006,False,11400.31,913,913,1105.23,12505.54,12.486648,12.486648,0.088379,51.757247
11,Catégorie_006,True,10626.19,2111,2076,14325.43,24951.62,5.033723,5.118589,0.574128,48.242753
13,Catégorie_007,True,9481.74,727,717,10318.29,19800.03,13.042283,13.224184,0.521125,18.184360
0,Catégorie_001,False,8761.19,236,285,0.00,8761.19,37.123686,30.741018,0.000000,92.143809
17,Unknown,True,7194.03,882,881,4672.72,11866.75,8.156497,8.165755,0.393766,12.523621


#### Store lens


In [17]:
from IPython.display import display

store_promo = summarize_promo_effect(analysis_base, ['Store_Province', 'Store_Name'])
display(store_promo.head(20))


,Store_Province,Store_Name,Promo_Flag,GMV,Units,Orders,Discount_value,Original_value,ASP,AOV,Avg_Discount_rate,GMV_share_pct
134,AB,Alberta Online Store 01,False,82425.32,3467,3306,0.00,82425.32,23.774249,24.932039,0.000000,100.000000
60,AB,AB Store 039 - Sherwood Park,False,4481.55,124,150,132.87,4614.42,36.141532,29.877000,0.028795,81.343556
38,AB,AB Store 026 - Medicine Hat,False,4403.61,132,146,122.38,4525.99,33.360682,30.161712,0.027039,85.446265
26,AB,AB Store 016 - Camrose,False,3282.76,83,111,117.72,3400.48,39.551325,29.574414,0.034619,80.062630
102,AB,AB Store 087 - Grande Prairie,False,3249.67,97,119,43.79,3293.46,33.501753,27.308151,0.013296,76.604858
56,AB,AB Store 037 - Chestermere,False,3118.83,73,112,81.82,3200.65,42.723699,27.846696,0.025564,83.686765
86,AB,AB Store 077 - Chestermere,False,3072.84,87,124,36.43,3109.27,35.320000,24.780968,0.011717,66.673610
116,AB,AB Store 097 - Chestermere,False,3035.79,83,104,33.39,3069.18,36.575783,29.190288,0.010879,78.962646
6,AB,AB Store 004 - Lethbridge,False,2958.37,93,103,68.04,3026.41,31.810430,28.722039,0.022482,70.666205
92,AB,AB Store 081 - Calgary,False,2956.03,103,123,44.68,3000.71,28.699320,24.032764,0.014890,75.057765


#### Promotion type lens


In [18]:
from IPython.display import display

promo_label_summary = summarize_promo_effect(analysis_base, ['Promo_Label'])
display(promo_label_summary.head(30))


,Promo_Label,Promo_Flag,GMV,Units,Orders,Discount_value,Original_value,ASP,AOV,Avg_Discount_rate,GMV_share_pct
8,OFFER11E623A3,False,200993.16,6826,7617,2686.04,203679.20,29.445233,26.387444,0.013188,100.0
18,OFFER3C5E0BCD,True,7619.68,661,658,7620.69,15240.37,11.527504,11.580061,0.500033,100.0
5,OFFER0B2B5288,True,7372.01,1017,1010,4876.74,12248.75,7.248781,7.299020,0.398142,100.0
61,OFFERF6059F7F,True,5282.15,1024,1022,4906.65,10188.80,5.158350,5.168444,0.481573,100.0
55,OFFERE7E4C9E9,True,2782.32,107,107,902.33,3684.65,26.002991,26.002991,0.244889,100.0
22,OFFER4CFF2747,True,1624.60,84,84,1410.72,3035.32,19.340476,19.340476,0.464768,100.0
41,OFFERB0603A29,True,1284.64,80,80,1139.24,2423.88,16.058000,16.058000,0.470007,100.0
13,OFFER1EFAA24C,True,1258.61,56,56,1258.59,2517.20,22.475179,22.475179,0.499996,100.0
60,OFFERF4AA4853,True,947.76,109,108,1756.79,2704.55,8.695046,8.775556,0.649568,100.0
10,OFFER176BF862,True,696.46,232,230,1796.99,2493.45,3.001983,3.028087,0.720684,100.0


#### Promo mix hot spots


In [19]:
from IPython.display import display

promo_combos = (
    analysis_base[analysis_base['Promo_Flag']]
    .groupby(['Category', 'Store_Name', 'Promo_Label'], as_index=False)
    .agg(
        GMV=('GMV', 'sum'),
        Units=('Quantity_item', 'sum'),
        Orders=('Transaction_ID', 'nunique'),
        Discount_value=('Discount_value', 'sum'),
        Original_value=('Original_value', 'sum'),
    )
    .assign(
        ASP=lambda df: df['GMV'] / df['Units'].replace({0: pd.NA}),
        AOV=lambda df: df['GMV'] / df['Orders'].replace({0: pd.NA}),
        Avg_Discount_rate=lambda df: df['Discount_value'] / df['Original_value'].replace({0: pd.NA}),
    )
    .sort_values('GMV', ascending=False)
    .head(30)
)

display(promo_combos)


,Category,Store_Name,Promo_Label,GMV,Units,Orders,Discount_value,Original_value,ASP,AOV,Avg_Discount_rate
175,Catégorie_003,AB Store 105 - St. Albert,OFFER3C5E0BCD,382.48,24,24,382.28,764.76,15.936667,15.936667,0.499869
955,Catégorie_007,AB Store 105 - St. Albert,OFFER3C5E0BCD,322.17,43,43,322.40,644.57,7.492326,7.492326,0.500178
39,Catégorie_002,AB Store 077 - Chestermere,OFFER3C5E0BCD,233.92,16,16,233.92,467.84,14.620000,14.620000,0.500000
517,Catégorie_006,AB Store 096 - Camrose,OFFERCD28EB23,201.92,11,2,1003.03,1204.95,18.356364,100.960000,0.832425
318,Catégorie_006,AB Store 016 - Camrose,OFFER0B2B5288,201.32,28,28,133.28,334.60,7.190000,7.190000,0.398326
81,Catégorie_002,AB Store 107 - Grande Prairie,OFFER3C5E0BCD,198.91,15,15,198.94,397.85,13.260667,13.260667,0.500038
243,Catégorie_004,AB Store 105 - St. Albert,OFFER3C5E0BCD,196.42,15,15,196.43,392.85,13.094667,13.094667,0.500013
78,Catégorie_002,AB Store 105 - St. Albert,OFFER3C5E0BCD,189.43,15,15,189.42,378.85,12.628667,12.628667,0.499987
154,Catégorie_003,AB Store 092 - Fort McMurray,OFFER3C5E0BCD,185.92,13,13,185.95,371.87,14.301538,14.301538,0.500040
331,Catégorie_006,AB Store 023 - Red Deer,OFFER0B2B5288,179.75,25,25,119.00,298.75,7.190000,7.190000,0.398326


**Observation:** Promotion cadence spikes ahead of summer and holiday periods; the `build_promotion_summary` table indicates promo baskets are significantly larger, implying room to refine targeting rather than increasing promo count.

## 5. Store performance

In [20]:
store_perf = build_profitability_by_store(sales_df).reset_index()
top_stores = store_perf.head(10)
top_stores


,LIBELLE_y,CITY,revenue,units,discount,avg_discount_per_unit
0,Alberta Online Store 01,Online,294051.44,3467,0.00,0.000000
1,AB Store 105 - St. Albert,St. Albert,37635.20,345,3438.18,9.965739
2,AB Store 039 - Sherwood Park,Sherwood Park,25916.82,237,975.44,4.115781
3,AB Store 107 - Grande Prairie,Grande Prairie,25574.15,217,1467.49,6.762627
4,AB Store 077 - Chestermere,Chestermere,25055.87,261,1641.91,6.290843
5,AB Store 081 - Calgary,Calgary,22300.21,218,1164.02,5.339541
6,AB Store 026 - Medicine Hat,Medicine Hat,22077.01,233,757.36,3.250472
7,AB Store 087 - Grande Prairie,Grande Prairie,21968.86,230,1225.29,5.327348
8,AB Store 004 - Lethbridge,Lethbridge,21027.81,207,953.43,4.605942
9,AB Store 083 - Red Deer,Red Deer,20315.15,188,917.43,4.879947


In [21]:
fig_store = px.bar(top_stores, x='LIBELLE_y', y='revenue', title='Top 10 Stores by Revenue')
fig_store.update_layout(xaxis_title='Store', yaxis_title='Revenue', xaxis_tickangle=45)
fig_store


### Store performance deep dive


In [22]:
from IPython.display import display

store_metrics = (
    sales_df.assign(
        Store_Name=sales_df['LIBELLE_y'].fillna(sales_df['Store_transaction_ID']),
        Store_City=sales_df['CITY'].fillna('Unknown'),
        Store_Province=sales_df['PROVINCE'].fillna('Unknown'),
    )
    .groupby(['Store_transaction_ID', 'Store_Name', 'Store_City', 'Store_Province'], as_index=False)
    .agg(
        GMV=('Transaction_value', 'sum'),
        Units=('Quantity_item', 'sum'),
        Orders=('Transaction_ID', 'nunique'),
    )
)

store_metrics['AOV'] = store_metrics['GMV'] / store_metrics['Orders'].replace({0: pd.NA})
store_metrics['Items_per_Order'] = store_metrics['Units'] / store_metrics['Orders'].replace({0: pd.NA})
store_metrics['GMV_share_pct'] = (store_metrics['GMV'] / store_metrics['GMV'].sum()) * 100
store_metrics = store_metrics.sort_values('GMV', ascending=False).reset_index(drop=True)
store_columns = ['Store_transaction_ID', 'Store_Name', 'Store_City', 'Store_Province', 'GMV', 'Units', 'Orders', 'AOV', 'Items_per_Order', 'GMV_share_pct']
top_stores_detail = store_metrics.loc[:, store_columns].head(15)
lagging_stores_detail = store_metrics.nsmallest(15, 'GMV').loc[:, store_columns]
display(top_stores_detail)
display(lagging_stores_detail)


,Store_transaction_ID,Store_Name,Store_City,Store_Province,GMV,Units,Orders,AOV,Items_per_Order,GMV_share_pct
0,ABWEB01,Alberta Online Store 01,Online,AB,294051.44,3467,3306,88.944779,1.048699,27.741752
1,AB105,AB Store 105 - St. Albert,St. Albert,AB,37635.20,345,377,99.828117,0.915119,3.550625
2,AB039,AB Store 039 - Sherwood Park,Sherwood Park,AB,25916.82,237,262,98.919160,0.904580,2.445076
3,AB107,AB Store 107 - Grande Prairie,Grande Prairie,AB,25574.15,217,242,105.678306,0.896694,2.412747
4,AB077,AB Store 077 - Chestermere,Chestermere,AB,25055.87,261,296,84.648209,0.881757,2.363851
5,AB081,AB Store 081 - Calgary,Calgary,AB,22300.21,218,236,94.492415,0.923729,2.103873
6,AB026,AB Store 026 - Medicine Hat,Medicine Hat,AB,22077.01,233,247,89.380607,0.943320,2.082816
7,AB087,AB Store 087 - Grande Prairie,Grande Prairie,AB,21968.86,230,251,87.525339,0.916335,2.072612
8,AB004,AB Store 004 - Lethbridge,Lethbridge,AB,21027.81,207,215,97.803767,0.962791,1.983831
9,AB083,AB Store 083 - Red Deer,Red Deer,AB,20315.15,188,198,102.601768,0.949495,1.916596


,Store_transaction_ID,Store_Name,Store_City,Store_Province,GMV,Units,Orders,AOV,Items_per_Order,GMV_share_pct
68,AB094,AB Store 094 - Canmore,Canmore,AB,1636.66,23,29,56.436552,0.793103,0.154408
67,ABWEB02,Alberta Online Store 02,Online,AB,2130.03,26,22,96.819545,1.181818,0.200954
66,AB034,AB Store 034 - Canmore,Canmore,AB,2690.02,24,30,89.667333,0.800000,0.253785
65,AB010,AB Store 010 - Leduc,Leduc,AB,2733.59,31,43,63.571860,0.720930,0.257896
64,AB052,AB Store 052 - Fort McMurray,Fort McMurray,AB,2782.61,27,33,84.321515,0.818182,0.262520
63,AB095,AB Store 095 - Brooks,Brooks,AB,3121.98,38,45,69.377333,0.844444,0.294538
62,AB100,AB Store 100 - Wetaskiwin,Wetaskiwin,AB,3302.63,35,43,76.805349,0.813953,0.311581
61,AB011,AB Store 011 - Okotoks,Okotoks,AB,4232.87,52,58,72.980517,0.896552,0.399342
60,AB014,AB Store 014 - Canmore,Canmore,AB,4583.69,44,48,95.493542,0.916667,0.432440
59,AB049,AB Store 049 - Spruce Grove,Spruce Grove,AB,4709.38,52,54,87.210741,0.962963,0.444298


### City / province lens


In [23]:
from IPython.display import display

geo_perf = (
    sales_df.assign(
        City=sales_df['CITY'].fillna('Unknown'),
        Province=sales_df['PROVINCE'].fillna('Unknown'),
    )
    .groupby(['Province', 'City'], as_index=False)
    .agg(
        GMV=('Transaction_value', 'sum'),
        Units=('Quantity_item', 'sum'),
        Orders=('Transaction_ID', 'nunique'),
    )
)

geo_perf['AOV'] = geo_perf['GMV'] / geo_perf['Orders'].replace({0: pd.NA})
geo_perf['Items_per_Order'] = geo_perf['Units'] / geo_perf['Orders'].replace({0: pd.NA})
geo_perf['GMV_share_pct'] = (geo_perf['GMV'] / geo_perf['GMV'].sum()) * 100
geo_perf = geo_perf.sort_values('GMV', ascending=False).reset_index(drop=True)

display(geo_perf.head(20))

city_category_mix = (
    sales_df.assign(
        City=sales_df['CITY'].fillna('Unknown'),
        Category=sales_df['Catégorie'].fillna('Unknown'),
    )
    .groupby(['City', 'Category'], as_index=False)['Transaction_value']
    .sum()
)

city_category_mix['GMV_share_pct'] = city_category_mix.groupby('City')['Transaction_value'].transform(
    lambda values: (values / values.sum()) * 100
)
city_category_top = (
    city_category_mix.sort_values(['City', 'Transaction_value'], ascending=[True, False])
    .groupby('City')
    .head(5)
)

display(city_category_top.head(25))


,Province,City,GMV,Units,Orders,AOV,Items_per_Order,GMV_share_pct
0,AB,Online,296181.47,3493,3328,88.996836,1.049579,27.942705
1,AB,Grande Prairie,83875.71,790,866,96.854169,0.912240,7.913102
2,AB,Chestermere,70641.83,707,817,86.464908,0.865361,6.664576
3,AB,St. Albert,62031.01,605,664,93.420196,0.911145,5.852204
4,AB,Calgary,59827.54,615,679,88.111252,0.905744,5.644321
5,AB,Medicine Hat,56513.58,541,577,97.943813,0.937608,5.331672
6,AB,Red Deer,55138.15,523,566,97.417226,0.924028,5.201909
7,AB,Lethbridge,48508.65,505,588,82.497704,0.858844,4.576461
8,AB,Sherwood Park,44293.91,449,512,86.511543,0.876953,4.178829
9,AB,Fort McMurray,37366.18,435,491,76.102200,0.885947,3.525245


,City,Category,Transaction_value,GMV_share_pct
5,Airdrie,Catégorie_006,9512.06,26.781514
6,Airdrie,Catégorie_007,8338.82,23.478219
7,Airdrie,Unknown,8090.17,22.778137
1,Airdrie,Catégorie_002,3870.77,10.898279
2,Airdrie,Catégorie_003,2807.76,7.905340
15,Banff,Unknown,6780.41,29.834779
13,Banff,Catégorie_006,6083.04,26.766251
14,Banff,Catégorie_007,4660.51,20.506914
9,Banff,Catégorie_002,2808.38,12.357276
8,Banff,Catégorie_001,787.15,3.463573


### Promotion uplift modelling

In [24]:
# Aggregate line-level rows to transaction-level metrics (value, quantity, promo flag)
transactions = prepare_transaction_level(sales_df)
transactions.head()


,Transaction_ID,Transaction_value,Quantity_item,Promo_Flag
0,TRX00076125,-112.93,-1,False
1,TRX0012136F,312.55,1,False
2,TRX001A3B1F,75.95,1,False
3,TRX001B4724,35.85,1,True
4,TRX00274501,-54.95,-1,False


- `Promo_Flag` coefficient = estimated lift in order value when a promotion is active (holding the intercept constant).
- `p_value` tells us if that lift is statistically significant (compare to 0.05).

In [25]:
# Fit a simple OLS model: Transaction_value = beta0 + beta1 * Promo_Flag
regression_results = run_promo_regression(transactions)
regression_results


,term,coef,std_err,t,p_value
0,Intercept,93.057325,1.003926,92.693387,0.000000e+00
1,Promo_Flag,-11.577848,1.667990,-6.941197,4.089395e-12


In [26]:
# Compare promo vs non-promo order values with a Welch's t-test
ab_results = ab_test_promo(transactions)
pd.Series(ab_results).to_frame('value')


,value
promo_mean,8.147948e+01
nonpromo_mean,9.305732e+01
lift,-1.157785e+01
t_stat,-7.366818e+00
p_value,1.877408e-13


- `promo_mean` vs `nonpromo_mean`: average basket sizes for each group.
- `lift` = promo_mean - nonpromo_mean (positive means promos drive larger orders).
- `p_value`: < 0.05 indicates the lift is statistically significant.

## 6. Customer engagement

In [27]:
web_share = web_account_share(contact_df)
web_share


Has_Web_Account
N    0.931211
Y    0.068789
Name: count, dtype: float64

In [28]:
web_share = web_account_share(contact_df)
web_share_df = web_share.rename_axis('Has_Web_Account').reset_index(name='Share')
fig_web = px.pie(web_share_df, names='Has_Web_Account', values='Share', title='Web Account Adoption')
fig_web

Fewer than 5% of contacts maintain web credentials — a major opportunity for loyalty sign-ups and digital marketing conversion.

## 7. Network expansion

In [29]:
opening_trend = store_opening_trend(store_df)
opening_trend.tail()


,Year,Openings
19,2020,2
20,2022,4
21,2023,1
22,2024,2
23,2025,1


In [30]:
fig_openings = px.bar(opening_trend, x='Year', y='Openings', title='Store Openings by Year')
fig_openings


Recent openings skew toward 2023–2024, aligning with the higher revenue lifts observed in newer Alberta markets.

## 10. Customer value (RFM analysis)


In [31]:
from IPython.display import display

rfm_reference_date = sales_df['Date_transaction'].max()

rfm_transactions = (
    sales_df.groupby(['Contact_ID', 'Transaction_ID'], as_index=False)
    .agg(
        Transaction_date=('Date_transaction', 'max'),
        GMV=('Line_Sales_Value', 'sum'),
    )
)

rfm_base = (
    rfm_transactions.groupby('Contact_ID', as_index=False)
    .agg(
        Recency=('Transaction_date', lambda x: (rfm_reference_date - x.max()).days),
        Frequency=('Transaction_ID', 'nunique'),
        Monetary=('GMV', 'sum'),
        Last_purchase=('Transaction_date', 'max'),
    )
)

overall_recency = (rfm_reference_date - sales_df['Date_transaction'].min()).days
rfm_base['Recency'] = rfm_base['Recency'].fillna(overall_recency)

display(rfm_base.head())


,Contact_ID,Recency,Frequency,Monetary,Last_purchase
0,,0,9444,191768.36,2023-12-31
1,CUST0019396E,291,1,5.16,2023-03-15
2,CUST00242E6F,79,1,7.19,2023-10-13
3,CUST0037A6B7,276,1,12.49,2023-03-30
4,CUST00601607,245,1,40.00,2023-04-30


In [32]:
recency_quantiles = rfm_base['Recency'].quantile([0.2, 0.4, 0.6, 0.8])
frequency_quantiles = rfm_base['Frequency'].quantile([0.2, 0.4, 0.6, 0.8])
monetary_quantiles = rfm_base['Monetary'].quantile([0.2, 0.4, 0.6, 0.8])

def r_score(recency):
    if recency <= recency_quantiles.iloc[0]:
        return 5
    if recency <= recency_quantiles.iloc[1]:
        return 4
    if recency <= recency_quantiles.iloc[2]:
        return 3
    if recency <= recency_quantiles.iloc[3]:
        return 2
    return 1

def fm_score(value, quantiles):
    if value <= quantiles.iloc[0]:
        return 1
    if value <= quantiles.iloc[1]:
        return 2
    if value <= quantiles.iloc[2]:
        return 3
    if value <= quantiles.iloc[3]:
        return 4
    return 5

rfm_base['R'] = rfm_base['Recency'].apply(r_score)
rfm_base['F'] = rfm_base['Frequency'].apply(lambda x: fm_score(x, frequency_quantiles))
rfm_base['M'] = rfm_base['Monetary'].apply(lambda x: fm_score(x, monetary_quantiles))
rfm_base['RFM_Score'] = rfm_base[['R', 'F', 'M']].astype(int).astype(str).agg(''.join, axis=1)

def map_segment(total):
    if total >= 12:
        return 'Champions'
    if total >= 9:
        return 'Loyal'
    if total >= 6:
        return 'Potentials'
    return 'At Risk'

rfm_base['RFM_Segment'] = rfm_base[['R', 'F', 'M']].sum(axis=1).apply(map_segment)

display(rfm_base.head())


,Contact_ID,Recency,Frequency,Monetary,Last_purchase,R,F,M,RFM_Score,RFM_Segment
0,,0,9444,191768.36,2023-12-31,5,5,5,555,Champions
1,CUST0019396E,291,1,5.16,2023-03-15,1,1,1,111,At Risk
2,CUST00242E6F,79,1,7.19,2023-10-13,4,1,2,412,Potentials
3,CUST0037A6B7,276,1,12.49,2023-03-30,2,1,3,213,Potentials
4,CUST00601607,245,1,40.00,2023-04-30,2,1,5,215,Potentials


In [33]:
rfm_summary = (
    rfm_base.groupby('RFM_Segment', as_index=False)
    .agg(
        Customers=('Contact_ID', 'nunique'),
        Avg_Recency=('Recency', 'mean'),
        Avg_Frequency=('Frequency', 'mean'),
        Avg_Monetary=('Monetary', 'mean'),
        GMV=('Monetary', 'sum'),
    )
)
rfm_summary['Customer_share_pct'] = (rfm_summary['Customers'] / rfm_summary['Customers'].sum()) * 100
rfm_summary['GMV_share_pct'] = (rfm_summary['GMV'] / rfm_summary['GMV'].sum()) * 100

display(rfm_summary.sort_values('GMV', ascending=False))


,RFM_Segment,Customers,Avg_Recency,Avg_Frequency,Avg_Monetary,GMV,Customer_share_pct,GMV_share_pct
1,Champions,23,68.260870,415.782609,8459.106957,194559.46,0.965170,81.453556
3,Potentials,1215,163.688066,1.000000,18.731893,22759.25,50.986152,9.528305
2,Loyal,572,55.575175,1.008741,31.546696,18044.71,24.003357,7.554533
0,At Risk,573,260.584642,1.000000,6.101152,3495.96,24.045321,1.463606


In [34]:
rfm_by_category = (
    sales_df.merge(rfm_base[['Contact_ID', 'RFM_Segment']], on='Contact_ID')
    .assign(Category=lambda df: df['Catégorie'].fillna('Unknown'))
    .groupby(['RFM_Segment', 'Category'], as_index=False)
    .agg(
        GMV=('Line_Sales_Value', 'sum'),
        Orders=('Transaction_ID', 'nunique'),
    )
)
rfm_by_category['GMV_share_pct'] = rfm_by_category.groupby('RFM_Segment')['GMV'].transform(
    lambda values: (values / values.sum()) * 100 if values.sum() else 0
)

display(rfm_by_category.sort_values(['RFM_Segment', 'GMV'], ascending=[True, False]).head(40))


,RFM_Segment,Category,GMV,Orders,GMV_share_pct
5,At Risk,Catégorie_006,1578.40,330,45.149258
6,At Risk,Catégorie_007,817.99,91,23.398151
7,At Risk,Unknown,795.08,126,22.742823
1,At Risk,Catégorie_002,143.18,13,4.095585
2,At Risk,Catégorie_003,53.98,4,1.544068
3,At Risk,Catégorie_004,49.47,5,1.415062
4,At Risk,Catégorie_005,47.87,3,1.369295
0,At Risk,Catégorie_001,9.99,1,0.285758
16,Champions,Unknown,47171.01,1997,24.245035
14,Champions,Catégorie_007,41529.29,2210,21.345295


In [35]:
recency_bins = [0, 30, 60, 90, 180, 365, float('inf')]
recency_labels = ['0-30 days', '31-60', '61-90', '91-180', '181-365', '365+']

rfm_base['Recency_bucket'] = pd.cut(rfm_base['Recency'], bins=recency_bins, labels=recency_labels, right=True)

recency_summary = (
    rfm_base.groupby('Recency_bucket', as_index=False)
    .agg(
        Customers=('Contact_ID', 'nunique'),
        GMV=('Monetary', 'sum'),
    )
)
recency_summary['Customer_share_pct'] = (recency_summary['Customers'] / recency_summary['Customers'].sum()) * 100

display(recency_summary.sort_values('Recency_bucket'))


/var/folders/_h/v3zx4l357d3fgz8svznhxcjc0000gn/T/ipykernel_30932/26847261.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Recency_bucket,Customers,GMV,Customer_share_pct
0,0-30 days,343,8044.56,14.472574
1,31-60,293,5645.71,12.362869
2,61-90,198,3514.51,8.354430
3,91-180,494,8853.62,20.843882
4,181-365,1042,20615.65,43.966245
5,365+,0,0.00,0.000000
